In [1]:
import pandas as pd
import numpy as np 
import datetime
import os

def recursive_file_gen(root_path):
    for root, dirs, files in os.walk(root_path):
        for file in files:
            yield os.path.join(root,file)
            
last_saturday=datetime.datetime.now().date()-datetime.timedelta(days=datetime.datetime.now().date().weekday()+2)

pos_list=list(recursive_file_gen("/home/jian/BigLots/"))
pos_list=[x for x in pos_list if "daily" in x.lower() and "s/MediaStorm_" in x and x[-4:]==".txt"]
pos_list=[x for x in pos_list if x.split("s/MediaStorm_")[1][:10]>="2019-02-03"]
pos_list.sort()
print(len(pos_list))



58


In [4]:
pos_list.pop(0)
len(pos_list)

57

In [5]:
file_0="/home/jian/BigLots/hist_daily_data_itemlevel_decompressed/MediaStormDailySalesHistory20190209.txt"
pos_list=[file_0]+pos_list
len(pos_list)

58

In [7]:
df_all_POS_by_store=pd.DataFrame()
for file in pos_list:
    df=pd.read_csv(file,dtype=str,sep="|")
    df['item_transaction_amt']=df['item_transaction_amt'].astype(float)
    df_rewards=df[pd.notnull(df['customer_id_hashed'])]
    df_non_rewards=df[pd.isnull(df['customer_id_hashed'])]
    
    df_rewards_sales=df_rewards.groupby(['location_id','transaction_dt'])['item_transaction_amt'].sum().to_frame().reset_index().rename(columns={"item_transaction_amt":"rewards_sales"})
    df_rewards_trans=df_rewards[['location_id','transaction_dt','transaction_id','customer_id_hashed']].drop_duplicates()
    df_rewards_trans['rewards_trans']=1
    df_rewards_trans=df_rewards_trans.groupby(['location_id','transaction_dt'])['rewards_trans'].sum().to_frame().reset_index()
    df_rewards=pd.merge(df_rewards_sales,df_rewards_trans,on=['location_id','transaction_dt'],how="outer")
    
    df_non_rewards_sales=df_non_rewards.groupby(['location_id','transaction_dt'])['item_transaction_amt'].sum().to_frame().reset_index().rename(columns={"item_transaction_amt":"non_rewards_sales"})
    df_non_rewards_trans=df_non_rewards[['location_id','transaction_dt','transaction_id']].drop_duplicates()
    df_non_rewards_trans['non_rewards_trans']=1
    df_non_rewards_trans=df_non_rewards_trans.groupby(['location_id','transaction_dt'])['non_rewards_trans'].sum().to_frame().reset_index()
    df_non_rewards=pd.merge(df_non_rewards_sales,df_non_rewards_trans,on=['location_id','transaction_dt'],how="outer")
    
    df=pd.merge(df_rewards,df_non_rewards,on=['location_id','transaction_dt'],how="outer")
    df['week_end_dt']=df['transaction_dt'].max()
    df_all_POS_by_store=df_all_POS_by_store.append(df)
    print(datetime.datetime.now(),df['transaction_dt'].max())
df_all_POS_by_store['store_type']=np.where(df_all_POS_by_store['location_id']=="6990","online","instore")
df_summary=df_all_POS_by_store.groupby(["week_end_dt",'store_type'])['rewards_sales','non_rewards_sales','rewards_trans','non_rewards_trans'].sum().reset_index()

df_summary.to_csv("/home/jian/celery/JT_new_sing_ups_rewards_rolling_table/output/sales_in_2019_JL_"+str(datetime.datetime.now())+".csv",index=False)


2020-03-19 11:52:09.392286 2019-02-09
2020-03-19 11:56:47.423480 2019-02-16
2020-03-19 12:01:48.335892 2019-02-23
2020-03-19 12:10:09.196111 2019-03-02
2020-03-19 12:17:18.151859 2019-03-09
2020-03-19 12:21:58.086629 2019-03-16
2020-03-19 12:26:32.306331 2019-03-23
2020-03-19 12:31:11.404429 2019-03-30
2020-03-19 12:38:19.937308 2019-04-06
2020-03-19 12:44:02.015311 2019-04-13
2020-03-19 12:46:20.646042 2019-04-20
2020-03-19 12:47:53.694556 2019-04-27
2020-03-19 12:53:06.979577 2019-05-04
2020-03-19 12:56:25.862281 2019-05-11
2020-03-19 13:00:30.081873 2019-05-18
2020-03-19 13:06:35.806531 2019-05-25
2020-03-19 13:10:34.418992 2019-06-01
2020-03-19 13:13:54.470284 2019-06-08
2020-03-19 13:18:41.626802 2019-06-15
2020-03-19 13:20:01.421687 2019-06-22
2020-03-19 13:22:48.639375 2019-06-29
2020-03-19 13:24:53.870463 2019-07-06
2020-03-19 13:27:02.647500 2019-07-13
2020-03-19 13:28:17.313556 2019-07-20
2020-03-19 13:29:05.590020 2019-07-27
2020-03-19 13:29:56.148579 2019-08-03
2020-03-19 1

In [8]:
os.getcwd()

'/home/jian/BL_weekly_crontab/cron_4_JT_rolling_rewards_tracker'